<a href="https://colab.research.google.com/github/Leonelts13/DispositivosMoviles/blob/main/Notebooks/Deber1/scrapyData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><strong>DATA EXTRACTION</strong></h1></center>


* __authors__ = [@Leonelts13](https://github.com/Leonelts13/DispositivosMoviles/tree/626578a8cfa60096e8d92dbf520c8c644ddda4c7/Notebooks/Deber1)
* __date__ = "20/12/2023"
* __version__ = "1.0.0"
* __license__ = "GPL"
* __title__ = "Extracción de datos de la Web (Scrapy)"
* __keywords__ = "Scrapy, Olympic Games, Medaltally, Data extraction"

# 1. Pre Requisitos

In [9]:
# Instalación de las librerias o dependencias necesarias
!pip install beautifulsoup4 requests pandas

In [315]:
# Load dependencies
import json # modulo para trabajar con archivos JSON
import requests # librería para hacer peticiones HTTP
import pandas as pd # librería para procesar datos, proporciona funciones para analíticas
from bs4 import BeautifulSoup as bs # Extraer información de páginas web


# 2. Funciones para procesamiento de los datos

In [316]:
# Función que convierte una lista de elementos en un dataframe de pandas

def rowsToDataFrame(rows):
  #Verificamos que las cabeceras se esten tomando correctamente
    #print("Cabeceras de la tabla:")
    #print(rows[0])

    df = pd.DataFrame(rows[1:], columns=rows[0])
    return df

In [317]:
# Función para extraer los datos de una tabla HTML
def processTableData(tbl):
    rows = []

    # Encuentra todas las filas en el cuerpo de la tabla
    for child in tbl.find('tbody').children:
        row = []

        # Si es una fila de cabecera (<th>), se extraen los textos o títulos directamente
        if child.name == 'tr' and child.find_all('th'):
            for th in child.find_all('th'):
                # Busca la etiqueta <a> dentro de la celda de cabecera
                a_tag = th.find('a')
                if a_tag:
                  #almacenamos el nombre de la cabecera que se encuentra en la etiqueta <a>, caso contrario le asignamos un espacio en blanco ' '
                    header_text = a_tag.get('title', ' ')
                else:
                  #almacenamos el nombre de la cabecera que se encuentra en la etiqueta <th>, caso contrario le asignamos un espacio en blanco ' '
                    header_text = th.text.replace('\n', '') if th.text else ' '
                row.append(header_text)
        else:
            # Si es una fila de datos (<td>), se realiza un proceso similar a la versión anterior
            for td in child:
                try:
                    item = td.text.replace('\n', '') # recupera el texto del elemento [<td>]
                    if item:
                        row.append(item) # Agrega cada elemento de la fila
                except:
                    continue

        if len(row) > 0:
            rows.append(row)

    # print(rows)
    return rows

In [318]:
# Función para extraer la data de la URL y procesar el contenido HTML
def processDataHTML(data):
  #Seleccionamos la tabla que vamos a usar, data['sections'][0]['text'] --> donde [0] indica la posicion en la que se encuentra la tabla a seleccionar
  #todo dependera del formato que se encuentre en "Inpeccionar elemento --> network --> Fetch --> archivo API"
    soup = bs(data['sections'][0]['text'], 'html.parser') # Recupera el item del JSON y lo convierte en contenido HTML
    tbl = soup.find_all('table')[0] # Recupera la tabla del contenido HTML

    #Imprimimos todo el formato de la tabla recuperada
    #print(tbl.prettify())

    tblRows = processTableData(tbl) # Ejecuta la función apra porcesar el contenido de la Tabla
    return tblRows

# 3. Extracción de Información

In [323]:
# Read website
#r = requests.get('https://www.wikiwand.com/es/Anexo:Medallero_de_Europa_en_los_Juegos_Ol%C3%ADmpicos')
r = requests.get('https://www.wikiwand.com/mcs-api/es.wikipedia.org/v1/page/mobile-sections-remaining/Anexo%3AMedallero_de_Europa_en_los_Juegos_Ol%C3%ADmpicos')  # Recuperamos el contenido de la web
# print(r)
table = processDataHTML(json.loads(r.content)) # Formate el contenido en formato JSON
#print(table)
df = rowsToDataFrame(table) # ejecuta la funcion para recupera los datos de la tabla como dataframe de Pandas
#df


In [324]:
# Reemplaza cabeceras vacías por un nombre no nulo (por ejemplo, 'TOTAL')
#df.columns = [col if col != ' ' else 'Total' for col in df.columns]
#df.columns = [col if col.strip() != '' else f'Total{i+1}' for i, col in enumerate(df.columns)]

# Reemplaza los nombres de las columnas con espacios en blanco
#Crea una lista llamada nombres_reemplazo que contiene los nuevos nombres para las columnas con espacios en blanco.
nombres_reemplazo = [f'Total_{i+1}' for i, col in enumerate(df.columns) if col.strip() == '']

#Utiliza una comprensión de listas para actualizar los nombres de las columnas en el DataFrame. Si el nombre de la columna tiene un espacio en blanco, se reemplaza por el nombre correspondiente de nombres_reemplazo.
df.columns = [f'Total_{i}' if col.strip() == '' else col for i, col in enumerate(df.columns)]


df

,País,N.º Juegos de verano,Oro,Plata,Bronce,Total_5,N.º Juegos de invierno,Oro,Plata,Bronce,Total_10,N° Juegos Total,Oro,Plata,Bronce,Total_15
0,Alemania (GER)[3][4],17,239,267,291,797,12,112,104,70,286,29,351,371,361,1083
1,Alemania Occidental (FRG)[3],5,56,67,81,204,7,11,15,13,39,12,67,82,94,243
2,Alemania Oriental (GDR)[3],5,153,129,127,409,6,39,36,35,110,11,192,165,162,519
3,Armenia (ARM),7,2,8,8,18,7,0,0,0,0,14,2,8,8,18
4,Atletas Olímpicos de Rusia (OAR)[5],0,0,0,0,0,1,2,6,9,17,1,2,6,9,17
5,Austria (AUT),28,22,37,43,102,24,71,88,91,250,52,93,125,134,352
6,Azerbaiyán (AZE),7,7,14,28,49,6,0,0,0,0,13,7,14,28,49
7,Bélgica (BEL),27,46,57,63,166,21,2,2,4,8,48,48,59,67,174
8,Bielorrusia (BLR),7,13,30,42,85,7,8,7,5,20,14,21,37,47,105
9,Bohemia (BOH)[6],3,0,1,3,4,0,0,0,0,0,3,0,1,3,4


In [325]:
# Inspeccionamos el tipo de dato de cada variable
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   País                    55 non-null     object
 1   N.º Juegos de verano    55 non-null     object
 2   Oro                     55 non-null     object
 3   Plata                   55 non-null     object
 4   Bronce                  55 non-null     object
 5   Total_5                 55 non-null     object
 6   N.º Juegos de invierno  55 non-null     object
 7   Oro                     55 non-null     object
 8   Plata                   55 non-null     object
 9   Bronce                  55 non-null     object
 10  Total_10                55 non-null     object
 11  N° Juegos Total         55 non-null     object
 12  Oro                     55 non-null     object
 13  Plata                   55 non-null     object
 14  Bronce                  55 non-null     object
 15  Total_15

# 4. Almacenamiento de los datos

In [326]:
# Guardamos las datos extraídos como archivo CSV
# se puede aplicar otras opciones para guardar (Base de datos)
df.to_csv('OlympicGames_Europe_MedalTally.csv')

# 5. Referencias

### DEPENDENCIES
* [beautifulsoup4](https://pypi.org/project/BeautifulSoup/)
* [requests](https://pypi.org/project/requests/)
* [pandas](https://pypi.org/project/pandas/)

### Urls de Interés
* [Medallero de Europa en los Juegos Olímpicos 1](https://www.wikiwand.com/es/Anexo:Medallero_de_Europa_en_los_Juegos_Ol%C3%ADmpicos)
